In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

%%time
%run ../py/sql_utlis.py

ZMQInteractiveShell_obj = get_ipython()
su = SqlUtilities()
_, CURSOR = su.get_jh_conn_cursor()
su.create_navigableparentsequence_table(CURSOR)
su.populate_navigableparentsequence_table(CURSOR)

Wall time: 7.06 s


In [5]:

sql_str = '''
    SELECT *
    FROM [Jobhunting].[dbo].[NavigableParentSequence] nps;'''
nps_df = pd.DataFrame(su.get_execution_results(CURSOR, sql_str, verbose=False))
nps_df

,navigable_parent_sequence_id,file_name_id,navigable_parent_id,sequence_order,mrs_id
0,0,0,277,0,0
1,1,0,12552,1,0
2,2,0,2273,2,0
3,3,0,12062,3,0
4,4,0,2086,4,0
...,...,...,...,...,...
31921,31921,531,13190,21,0
31922,31922,531,2035,22,0
31923,31923,531,11972,23,0
31924,31924,531,1243,24,0


In [16]:

sequence_dict = {}
def f(df):
    file_name_id = df.file_name_id.tolist()[0]
    sequence_dict[file_name_id] = df.set_index('navigable_parent_sequence_id').sort_values('sequence_order')
nps_df.groupby('file_name_id').apply(f)

""


In [33]:

su.create_minimumrequirementssection_table(CURSOR)
su.populate_minimumrequirementssection_table(CURSOR)
sql_str = '''
    SELECT *
    FROM [Jobhunting].[dbo].[MinimumRequirementsSection] mrs;'''
mrs_df = pd.DataFrame(su.get_execution_results(CURSOR, sql_str, verbose=False)).set_index('mrs_id')
mrs_df.mrs_symbol.to_dict()

{0: 'N', 1: 'B', 2: 'M', 3: 'E'}

In [24]:

import random
import textwrap

file_name_id = random.sample(list(sequence_dict), 1)[0]
df = sequence_dict[file_name_id]
navigable_parent_id = random.sample(df.navigable_parent_id.tolist(), 1)[0]
sql_str = f'''
    SELECT np.[navigable_parent]
    FROM [Jobhunting].[dbo].[NavigableParents] np
    WHERE np.[navigable_parent_id] = {navigable_parent_id};'''
html_str = pd.DataFrame(su.get_execution_results(CURSOR, sql_str, verbose=False)).navigable_parent.squeeze()
print('\n# ' + '\n# '.join(textwrap.wrap(html_str)))


# <li>Experience analyzing the multiple facets of next generation
# sequencing data, including amplicon sequencing, tag counting, exon
# sequencing</li>


In [50]:

def get_dictionary_code():
    file_name_id = random.sample(list(sequence_dict), 1)[0]
    df = sequence_dict[file_name_id]
    output_str = ''
    for navigable_parent_sequence_id, row_series in df.iterrows():
        navigable_parent_id = row_series.navigable_parent_id
        sql_str = f'''
            SELECT np.[navigable_parent]
            FROM [Jobhunting].[dbo].[NavigableParents] np
            WHERE np.[navigable_parent_id] = {navigable_parent_id};'''
        html_str = pd.DataFrame(su.get_execution_results(CURSOR, sql_str)).navigable_parent.squeeze()
        output_str += '\n# ' + '\n# '.join(textwrap.wrap(html_str))
        output_str += "\n# {0: 'N', 1: 'B', 2: 'M', 3: 'E'}"
        mrs_id = row_series.mrs_id
        if mrs_id == 0:
            mrs_id = 'xx'
        output_str += f'\nsql_str = r"""UPDATE NavigableParentSequence\n    SET\n        mrs_id = {mrs_id}\n'
        output_str += f'    WHERE (navigable_parent_sequence_id = {navigable_parent_sequence_id})"""\n'
        output_str += r"cursor_obj = CURSOR.execute(sql_str)"
        output_str += '\nCURSOR.commit()\n'
    
    return output_str

In [ ]:

ZMQInteractiveShell_obj.set_next_input(text=get_dictionary_code(), replace=True)